In [2]:
out_dir = "017_vs_018"

In [2]:
# 対戦は旧特徴量、棋譜は新特徴量
# https://github.com/Lgeu/hungry_geese/commit/fb112ddf47330d7e0b26e8ad3fa4d95d53176522
!g++ ../../../KKT89/Combine.cpp -O3 -Ofast -march=native -std=c++17

In file included from ../../../KKT89/src/Goose.hpp:3,
                 from ../../../KKT89/src/Stage.hpp:4,
                 from ../../../KKT89/src/Duct.hpp:7,
                 from ../../../KKT89/src/Duct.cpp:2,
                 from ../../../KKT89/Combine.cpp:3:
../../../KKT89/src/Evaluator.hpp: In member function ‘void hungry_geese::evaluation_function::BitBoard::ShiftRight()’:
../../../KKT89/src/library.hpp:64:9: warning: format ‘%d’ expects argument of type ‘int’, but argument 3 has type ‘hungry_geese::evaluation_function::BitBoard::ull’ {aka ‘long long unsigned int’} [-Wformat=]
   64 |         "`%s` (%d) is out of range [%d, %d)", #value, value, left, right)
      |         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
../../../KKT89/src/library.hpp:54:24: note: in definition of macro ‘ASSERT’
   54 |                 printf(__VA_ARGS__); \
      |                        ^~~~~~~~~~~
../../../KKT89/src/Evaluator.hpp:582:9: note: in expansion of macro ‘ASSERT_RANGE’
  582 |         ASSERT_

../../../KKT89/src/Evaluator2.hpp: In member function ‘void hungry_geese::evaluation_function_2::BitBoard::ShiftRight()’:
../../../KKT89/src/library.hpp:64:9: warning: format ‘%d’ expects argument of type ‘int’, but argument 3 has type ‘hungry_geese::evaluation_function_2::BitBoard::ull’ {aka ‘long long unsigned int’} [-Wformat=]
   64 |         "`%s` (%d) is out of range [%d, %d)", #value, value, left, right)
      |         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
../../../KKT89/src/library.hpp:54:24: note: in definition of macro ‘ASSERT’
   54 |                 printf(__VA_ARGS__); \
      |                        ^~~~~~~~~~~
../../../KKT89/src/Evaluator2.hpp:582:9: note: in expansion of macro ‘ASSERT_RANGE’
  582 |         ASSERT_RANGE(lo, 0ull, 1ull << 55);
      |         ^~~~~~~~~~~~
../../../KKT89/src/library.hpp:64:17: note: format string is defined here
   64 |         "`%s` (%d) is out of range [%d, %d)", #value, value, left, right)
      |                ~^
      |             

In file included from ../../../KKT89/src/Duct.hpp:10,
                 from ../../../KKT89/src/Duct.cpp:2,
                 from ../../../KKT89/Combine.cpp:3:
../../../KKT89/src/Evaluator.hpp: In member function ‘void hungry_geese::evaluation_function::nn::Model<in_dim, out_dim, hidden_1, hidden_2>::LoadParameters(const char*) [with int in_dim = 2382; int out_dim = 5; int hidden_1 = 256; int hidden_2 = 32]’:
../../../KKT89/src/Evaluator.hpp:409:14: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  409 |         fread(&embed.parameters.weight, embed.parameters.weight.Ravel().size(), sizeof(short), fp);
      |         ~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
../../../KKT89/src/Evaluator.hpp:410:14: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  410 |  

In [7]:
!ls ../../rl/parameters

013_01.bin  017_02.bin	018_01.bin


In [13]:
!mkdir {out_dir}
!mkdir {out_dir}/1vs3
!mkdir {out_dir}/2vs2
!mkdir {out_dir}/3vs1

In [10]:
# コマンド確認
param1 = "../../rl/parameters/017_02.bin"
param2 = "../../rl/parameters/018_01.bin"
!./a.out -j -o ./ -t 0.1 -p {param1} {param2} {param1} {param2}

In [14]:
from subprocess import Popen
N_MATCHES = 50000
p1 = Popen(f"seq 1 {N_MATCHES} | xargs -I@ -P 20 sh -c './a.out -j -o {out_dir}/1vs3/ -t 0.1 -p {param1} {param2} {param2} {param2}'", shell=True)
p2 = Popen(f"seq 1 {N_MATCHES} | xargs -I@ -P 20 sh -c './a.out -j -o {out_dir}/2vs2/ -t 0.1 -p {param1} {param1} {param2} {param2}'", shell=True)
p3 = Popen(f"seq 1 {N_MATCHES} | xargs -I@ -P 20 sh -c './a.out -j -o {out_dir}/3vs1/ -t 0.1 -p {param1} {param1} {param1} {param2}'", shell=True)

In [3]:
!ls -1 {out_dir}/1vs3 | wc -l
!ls -1 {out_dir}/2vs2 | wc -l
!ls -1 {out_dir}/3vs1 | wc -l

6350
6612
6451


In [4]:
import numpy as np

ranks_param2 = []
files = !ls -1 {out_dir}/3vs1
for file in files:
    ranks = !tail {out_dir}/3vs1/{file} -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param2.append(ranks[-1])
rank_count = np.bincount(ranks_param2)[1:]
print("3vs1", rank_count, rank_count / rank_count.sum())

ranks_param2 = []
files = !ls -1 {out_dir}/2vs2
for file in files:
    ranks = !tail {out_dir}/2vs2/{file} -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param2.append(ranks[2])
    ranks_param2.append(ranks[3])
rank_count = np.bincount(ranks_param2)[1:]
print("2vs2", rank_count, rank_count / rank_count.sum())

ranks_param2 = []
files = !ls -1 {out_dir}/1vs3
for file in files:
    ranks = !tail {out_dir}/1vs3/{file} -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param2.append(ranks[1])
    ranks_param2.append(ranks[2])
    ranks_param2.append(ranks[3])
rank_count = np.bincount(ranks_param2)[1:]
print("1vs3", rank_count, rank_count / rank_count.sum())

3vs1 [1234 1152 2186 1879] [0.19128817 0.17857696 0.33886219 0.29127267]
2vs2 [3745 2895 3544 3040] [0.28319722 0.21892015 0.26799758 0.22988506]
1vs3 [5850 4575 4620 4005] [0.30708661 0.24015748 0.24251969 0.21023622]


In [5]:
# 当然かもしれないけど AAAB だと行動が読めるので A が有利
# ナッシュ均衡のやつ…できるんだろうか

In [6]:
!zip -r -q 017_vs_018.zip {out_dir}